In [10]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError, Field

import json 

from typing import Annotated, List
from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

In [11]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com/")

kernel = Kernel()


chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
  
)
kernel.add_service(chat_completion_service)

In [12]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [13]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Get prompt execution settings and configure response format
settings = kernel.get_prompt_execution_settings_from_service_id("default")
assert isinstance(settings, OpenAIChatPromptExecutionSettings)
settings.response_format = TravelPlan

agent = ChatCompletionAgent(
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [20]:
from IPython.display import display, HTML


async def main():
    # Define the chat history
    chat_history = ChatHistory()

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 2 kids from Singapore to Melboune",
    ]

    for user_input in user_inputs:
        # Add the user input to the chat history
        chat_history.add_user_message(user_input)

        # Start building HTML output
        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""

        # Collect the agent's response
        async for content in agent.invoke_stream(chat_history):
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name
            # Add this line to collect the response content
            if hasattr(content, 'content') and content.content is not None:
                full_response += content.content

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate_json(full_response)

            # Display the validated model as formatted JSON
            formatted_json = json.dumps(travel_plan.model_dump(), indent=2)
            html_output += f"<div style='margin-bottom:20px'>"
            html_output += f"<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += f"</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += f"<div style='margin-bottom:20px; color:red;'>"
            html_output += f"<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += f"</div>"
            # Add this to see what the response contains for debugging
            html_output += f"<div style='margin-bottom:20px;'>"
            html_output += f"<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
            html_output += f"</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()